## Set Up Dependencies and Data


In [1]:
import random

import alifedata_phyloinformatics_convert as apc
from hstrat import hstrat
from hsurf import hsurf
import joblib
import more_itertools as mit
import pandas as pd

from pylib._make_hamming_distance_matrix import make_hamming_distance_matrix


In [2]:
df = pd.read_csv("https://osf.io/u89ft/download")


## Reproducibility


In [3]:
%load_ext watermark
%watermark -iwbmuvg -iv


Last updated: 2024-09-03T03:55:13.457363+00:00

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.20.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.5.0-1025-azure
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit

Git hash: 3a4ffa6ab50daad503c8aff90326305d6e500c64

Git branch: master

pandas                            : 1.5.3
hstrat                            : 1.11.4
hsurf                             : 0.3.0
alifedata_phyloinformatics_convert: 0.16.2
joblib                            : 1.3.2
more_itertools                    : 10.2.0

Watermark: 2.4.3



In [4]:
df.head()


,bitfield
0,61067863028768134243186439757
1,19598090090116010358700834409
2,72209332835900027321038445430
3,10004054782569452583966348625
4,36310280622434712054702068598


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   bitfield  9 non-null      object
dtypes: object(1)
memory usage: 204.0+ bytes


In [6]:
df.describe()


,bitfield
count,9
unique,9
top,61067863028768134243186439757
freq,1


In [7]:
joblib.hash(df)


'562762d857f6984802d39ce8a4ed9e1f'

## Data Prep


In [8]:
exclude_leading = 16

df["bitfield"] = df["bitfield"].apply(int)
df["bitfield value bitlengths"] = df["bitfield"].apply(int.bit_length)
df["bitfield wordlengths"] = (df["bitfield value bitlengths"] + 31) // 32
assert mit.one(df["bitfield wordlengths"].unique()) == 3
df["bitfield bitlengths"] = df["bitfield wordlengths"] * 32
df["surface bitlengths"] = df["bitfield bitlengths"] - exclude_leading
df["surface bytelengths"] = df["surface bitlengths"] // 8

df


,bitfield,bitfield value bitlengths,bitfield wordlengths,bitfield bitlengths,surface bitlengths,surface bytelengths
0,61067863028768134243186439757,96,3,96,80,10
1,19598090090116010358700834409,94,3,96,80,10
2,72209332835900027321038445430,96,3,96,80,10
3,10004054782569452583966348625,94,3,96,80,10
4,36310280622434712054702068598,95,3,96,80,10
5,34143885553685296573628601206,95,3,96,80,10
6,49927616333954097311855502331,96,3,96,80,10
7,39405130722394847844858160123,95,3,96,80,10
8,20836025406695731973441246070,95,3,96,80,10


In [9]:
bitfield_bitlength = int(mit.one(df["bitfield bitlengths"].unique()))
surface_mask = (  # mask off leading 16 bit
    1 << (bitfield_bitlength - exclude_leading)
) - 1
assert surface_mask.bit_count() == bitfield_bitlength - exclude_leading
df["bitfield surface"] = df["bitfield"].values & surface_mask

df


,bitfield,bitfield value bitlengths,bitfield wordlengths,bitfield bitlengths,surface bitlengths,surface bytelengths,bitfield surface
0,61067863028768134243186439757,96,3,96,80,10,184176754756112078665293
1,19598090090116010358700834409,94,3,96,80,10,193628343256807539015273
2,72209332835900027321038445430,96,3,96,80,10,193630318226715838552950
3,10004054782569452583966348625,94,3,96,80,10,193625258396163272742225
4,36310280622434712054702068598,95,3,96,80,10,193630309324792402072438
5,34143885553685296573628601206,95,3,96,80,10,193630309324792402072438
6,49927616333954097311855502331,96,3,96,80,10,188909689527025665139707
7,39405130722394847844858160123,95,3,96,80,10,193632056009895310353403
8,20836025406695731973441246070,95,3,96,80,10,188905637598147380302710


In [10]:
df.dtypes


bitfield                     object
bitfield value bitlengths     int64
bitfield wordlengths          int64
bitfield bitlengths           int64
surface bitlengths            int64
surface bytelengths           int64
bitfield surface             object
dtype: object

## Deserialize Columns


In [11]:
surface_bytelength = int(mit.one(df["surface bytelengths"].unique()))
print(f"{surface_bytelength=}")
site_selection_algo = hsurf.tilted_sticky_algo
differentia_bitwidth = 1

hstrat_columns = [
    hsurf.col_from_surf_int(
        value=value,
        differentia_bit_width=differentia_bitwidth,
        site_selection_algo=site_selection_algo,
        differentiae_byte_bit_order="little",
        num_strata_deposited_byte_width=2,  # u16
        num_strata_deposited_byte_order="little",
        value_byte_width=surface_bytelength,
    )
    for value in df["bitfield surface"].values
]


surface_bytelength=10


In [12]:
for col in hstrat_columns:
    print(col.GetNumStrataDeposited())


39
41
41
41
41
41
40
41
40


## Reconstruct Tree


In [13]:
tree_df = hstrat.build_tree(
    hstrat_columns,
    hstrat.__version__,
    force_common_ancestry=True,
)


## Surface simulation tree


In [14]:
print(apc.RosettaTree(tree_df).as_dendropy.as_ascii_plot(plot_metric="length"))


                                                                          /-- 5
                                                        /-----------------+    
                                   /--------------------+                 \-- 4
                                   |                    |                      
      /----------------------------+                    \------------------ 8  
      |                            |                                           
/-----+                            \----------------------------------------- 2
|     |                                                                        
|     |                                                                   / 6  
+     \-------------------------------------------------------------------+    
|                                                                         \-- 7
|                                                                              
+---------------------------------------

## Random tree


In [15]:
dummy = [random.randint(0, 2**80) for _ in range(9)]
print(make_hamming_distance_matrix(dummy).upgma_tree().as_ascii_plot())


                                                         /------------------- 0
                   /-------------------------------------+                     
                   |                                     \------------------- 4
/------------------+                                                           
|                  |                  /-------------------------------------- 8
|                  \------------------+                                        
|                                     |                  /------------------- 6
+                                     \------------------+                     
|                                                        \------------------- 7
|                                                                              
|                  /--------------------------------------------------------- 5
\------------------+                                                           
                   |                  /-